In [1]:
import re                  
import numpy as np         
import tensorflow as tf    
import os
import glob

# 11-7. 프로젝트: 멋진 작사가 만들기

- [[참고]](https://machinelearningmastery.com/how-to-develop-a-word-level-neural-language-model-in-keras/)

- [[Text Generation with miniature GPT]](https://keras.io/examples/generative/text_generation_with_miniature_gpt/)

## 데이터 읽어오기 

In [2]:
txt_file_path = os.getenv('HOME')+'/aiffel//lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:9])

데이터 크기: 187088
Examples:
 ['The Cat in the Hat', 'By Dr. Seuss', 'The sun did not shine.', 'It was too wet to play.', 'So we sat in the house', 'All that cold cold wet day.', 'I sat there with Sally.', 'We sat there we two.', 'And I said How I wish']


## 데이터 정제

---

- 추가로 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거합니다. 문장을 토큰화 했을 때 **토큰의 개수가 15개**를 넘어가면 잘라내기를 권합니다.

In [3]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

The Cat in the Hat
By Dr. Seuss
The sun did not shine.
It was too wet to play.
So we sat in the house
All that cold cold wet day.
I sat there with Sally.
We sat there we two.
And I said How I wish
We had something to do!


### 토큰화 
- 문장을 일정한 기준으로 쪼개기 (띄어쓰기를 기준으로) 
- 문장 부호 양쪽에 공백 추가 , 소문자로 변환 , 특수문자  제거 

In [4]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       
    # 소문자로 바꾸고 양쪽 공백을 삭제

    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        
    # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  
    # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  
    # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      
    # 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


#### 소스 문장(Source Sentence) = X_train
#### 타겟 문장(Target Sentence) = y_train
- 토큰화를 진행한 후 끝 단어 를 없애면 소스 문장, 첫 단어 를 없애면 타겟 문장

In [5]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
        
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> the cat in the hat <end>',
 '<start> by dr . seuss <end>',
 '<start> the sun did not shine . <end>',
 '<start> it was too wet to play . <end>',
 '<start> so we sat in the house <end>',
 '<start> all that cold cold wet day . <end>',
 '<start> i sat there with sally . <end>',
 '<start> we sat there we two . <end>',
 '<start> and i said how i wish <end>',
 '<start> we had something to do ! <end>']

In [6]:
len(corpus)

175749

## 평가 데이터셋 분리
---
- `tokenize()` 함수로 데이터를 Tensor로 변환한 후, 
- sklearn 모듈의 `train_test_split()` 함수를 사용해 훈련 데이터와 평가 데이터를 분리하도록 하겠습니다. 
- **단어장의 크기는 12,000 이상**으로 설정하세요! 
- 총 **데이터의 20% 를 평가 데이터셋**으로 사용해 주세요!

In [7]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 전체 단어 개수 
        filters=' ',    
        oov_token="<unk>"
    )
    tokenizer.fit_on_texts(corpus) 
    # corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   
    # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)
    # padding 뒤에 , 토큰 개수 15개

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2    6  903 ...    0    0    0]
 [   2  122 2584 ...    0    0    0]
 [   2    6  304 ...    0    0    0]
 ...
 [ 673   27    6 ...    6  189    3]
 [   2  673   27 ...    0    0    0]
 [   2  673   27 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f447269e7d0>


- `num_words`: 단어 빈도에 따른 사용할 단어 개수의 최대값. 가장 빈번하게 사용되는 num_words개의 단어만 보존합니다.
- `filters`: 문자열로, 각 성분이 텍스트에서 걸러진 문자에 해당됩니다. 디폴트 값은 모든 구두점이며, 거기에 탭과 줄 바꿈은 추가하고 ' 문자는 제외합니다.
- `lower`: 불리언. 텍스트를 소문자로 변환할지 여부.
- `split`: 문자열. 단어 분해 용도의 분리기.
- `char_level`: 참인 경우 모든 문자가 토큰으로 처리됩니다.
- `oov_token`: 값이 지정된 경우, text_to_sequence 호출 과정에서 단어색인(word_index)에 추가되어 어휘목록 외 단어를 대체합니다.

In [8]:
print(tensor[:3, :10])

[[   2    6  903   14    6 1350    3    0    0    0]
 [   2  122 2584   20    1    3    0    0    0    0]
 [   2    6  304  166   70  559   20    3    0    0]]


In [9]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


### Tartget / Source 분리 

In [10]:
src_input = tensor[:, :-1]  
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 
# 마지막 토큰은 <END>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    
# tensor에서 <START>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[   2    6  903   14    6 1350    3    0    0    0    0    0    0    0]
[   6  903   14    6 1350    3    0    0    0    0    0    0    0    0]


### 데이터셋 분리 

In [11]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=42)


print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140599, 14)
Target Train: (140599, 14)


## 인공지능 만들기

- 모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계하세요! 
- (Loss는 아래 제시된 Loss 함수를 그대로 사용!)

In [12]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.drop_1 = tf.keras.layers.Dropout(0.3)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.drop_1(out)
        out = self.linear(out)
        
        return out
    
    
# embedding_size = 256
# hidden_size = 1024

embedding_size = 1020
hidden_size = 2048

/
vocab_size = tokenizer.num_words + 1

model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

-  `vocab_size` : Embedding 레이어에 어휘 크기를 지정할 때 실제 어휘보다 1 더 크게 지정합니다. (12001)

- `embedding_size` : 워드 벡터의 차원수

- `hidden_size` : LSTM 레이어의 hidden state 의 차원수

임베딩 레이어의 크기는 각 단어가 나타내는 벡터의 길이입니다. 
이것은 일반적으로 100-500 사이의 영역에 있습니다. 
즉, 임베딩 레이어 크기가 250 인 경우 각 단어는 250- 길이 벡터로 표현됩니다.

#### LSTM 히든 레이어 크기
- 일반적으로 임베딩 레이어 출력의 크기를 LSTM 셀의 히든 레이어 수와 일치시킵니다. 
- LSTM 셀의 숨겨진 계층이 어디에서 왔는지 궁금 할 것입니다. 
- 셀의 각  sigmoid,  tanh 또는  은닉 상태 레이어는 실제로 노드의 집합이며, 그 수는 은닉 레이어  크기와 같습니다. 

In [13]:
optimizer = tf.keras.optimizers.Adam()

loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)


model.compile(loss=loss, optimizer=optimizer)

In [14]:
model.fit(enc_train, dec_train, validation_data=(enc_val, dec_val), epochs=10)

Epoch 1/10
4394/4394 [==============================] - 572s 130ms/step - loss: 3.1079 - val_loss: 2.7861
Epoch 2/10
4394/4394 [==============================] - 587s 134ms/step - loss: 2.5833 - val_loss: 2.5265
Epoch 3/10
4394/4394 [==============================] - 587s 134ms/step - loss: 2.1988 - val_loss: 2.3824
Epoch 4/10
4394/4394 [==============================] - 589s 134ms/step - loss: 1.8963 - val_loss: 2.3078
Epoch 5/10
4394/4394 [==============================] - 575s 131ms/step - loss: 1.6782 - val_loss: 2.2833
Epoch 6/10
4394/4394 [==============================] - 531s 121ms/step - loss: 1.5270 - val_loss: 2.2936
Epoch 7/10
4394/4394 [==============================] - 567s 129ms/step - loss: 1.4271 - val_loss: 2.3041
Epoch 8/10
4394/4394 [==============================] - 585s 133ms/step - loss: 1.3611 - val_loss: 2.3332
Epoch 9/10
4394/4394 [==============================] - 544s 124ms/step - loss: 1.3188 - val_loss: 2.3527
Epoch 10/10
4394/4394 [=======================

In [15]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   
        # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <END>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [16]:
generate_text(model, tokenizer, init_sentence="<start>  i love", max_len=20)

'<start> i love you , i love you <end> '

##  10 Epoch에서 val_loss 값을 2.2 목표 

- Embedding size 와 hidden size를 늘려가면서 학습시키기 


#### 1. val_loss: 2.6070

```python 
embedding_size = 600
hidden_size = 3000
```
- '<start> i love you , i m not gonna crack <end> '



#### 2. ResourceExhaustedError


```python
embedding_size = 1500
hidden_size = 5000
```
- 학습 실패 :  ResourceExhaustedError: OOM when allocating tensor with shape[20000,5000]


#### 3. val_loss: 2.6012


```python
embedding_size = 600
hidden_size = 2000
```
- '<start> i love it when you call me big poppa <end> '
    
#### 4. val_loss: 2.5906


```python
embedding_size = 600
hidden_size = 1500
```
- '<star t> i love you mom , you always be my favorite girl . <end> '
   
     
#### 5. ResourceExhaustedError


```python
embedding_size = 1200
hidden_size = 2400
```
- 학습 실패 :  ResourceExhaustedError: OOM when allocating tensor with shape[20000,5000]

    
#### 6. loss: 1.0426 - val_loss: 2.5855


```python
embedding_size = 726
hidden_size = 1800
```
- '<start> i love you , liberian girl <end> '
 

- embedding과 hidden size만 조정해서는 5 epoch 이후에는  다시 validation loss값이 늘어난다. 
- drop out layer를 추가해서 과적합 해결 시도 
- drop out (0.2) - epoch 6에서 다시 loss값 증가
```python 
Epoch 5/10
4394/4394 [==============================] - 443s 101ms/step - loss: 1.5711 - val_loss: 2.2955
Epoch 6/10
4394/4394 [==============================] - 443s 101ms/step - loss: 1.4105 - val_loss: 2.3132
```

- drop out (0.3) - epoch 6에서 다시 loss값 증가
- 10 epoch 에서 loss: 1.2750 - val_loss: 2.3837
```python
Epoch 5/10
4394/4394 [==============================] - 440s 100ms/step - loss: 1.6930 - val_loss: 2.2945
Epoch 6/10
4394/4394 [==============================] - 440s 100ms/step - loss: 1.5342 - val_loss: 2.2952
```